# Model Training
Colab version.

***
Train control panel

In [1]:
batch_size = 32
gt_size = 256
epochs = 100
learning_rate = 0.001
appendLog = True

mode = 'save' # Either: save, load, experiment
preprocess = 'Rgb' # Either: Rgb, Lum
architecture = 'Paper' # Either: Paper, 1x1

***
Environment Setup

In [ ]:
!pip install tensorflow_io
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
sys.path.insert(0,'/content/drive/MyDrive/Neural_Upscale_2x')
from Utilities import *
from Neural_Upscale_2x import *

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
data_path = '/content/drive/MyDrive/Neural_Upscale_2x/data'
save_path = '/content/drive/MyDrive/Neural_Upscale_2x/{}_{}'.format(preprocess, architecture)

***
Import Data (Raw train e val set, Ground truth resolution)

In [ ]:
train_path = data_path + '/train'
val_path = data_path + '/val'
test_path = data_path + '/test'

train_gt = image_dataset_from_directory(
    train_path, label_mode=None, image_size=(gt_size, gt_size), batch_size=batch_size)

val_gt = image_dataset_from_directory(
    val_path, label_mode=None, image_size=(gt_size, gt_size), batch_size=batch_size)

***
Data Preprocessing

In [ ]:
if preprocess == 'Rgb':
    routine = preprocess_dataset_rgb_model
    channels = 3
if preprocess == 'Lum':
    routine = preprocess_dataset_luminance_model
    channels = 1

train = routine(train_gt, training_gt_size=gt_size)
val = routine(val_gt, training_gt_size=gt_size)

***
Model Spec.

In [ ]:
depth_to_space_lambda_layer = tf.keras.layers.Lambda(lambda x:tf.nn.depth_to_space(x, 2))

if architecture == 'Paper':
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape= (None, None, channels)),
        tf.keras.layers.Conv2D(64, (5,5), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(channels * (2 ** 2), (3,3), activation='relu', padding='same'),  # (channels) *  (factor of 2) ** 2
        depth_to_space_lambda_layer
        ])

elif architecture == '1x1':
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape= (None, None, channels)),
        tf.keras.layers.Conv2D(64, (5,5), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(channels * (2 ** 2), (1,1), activation='relu', padding='same'),
        depth_to_space_lambda_layer
        ]) 
    
model.summary()

***
Model Train

In [ ]:
log_path = save_path + '/' + 'log.csv'
save_best_path = save_path + '/best_weights.hdf5'

if mode == 'save':

    # Model compile
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=inverse_PSNR)

    # Create save dir
    if not os.path.exists(save_path): os.mkdir(save_path)

    # Callbacks
    csv_logger = tf.keras.callbacks.CSVLogger(log_path, append=appendLog)
    save_best = tf.keras.callbacks.ModelCheckpoint(save_best_path, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

    # Fit
    history = model.fit(train, epochs=10, validation_data=val, verbose=2, callbacks=[csv_logger, save_best, reduce_lr])
    model.save(save_path)

if mode == 'load':
    model = tf.keras.models.load_model(save_path, compile = False)
    model.load_weights(save_best_path)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=inverse_PSNR)

if mode == 'experiment':
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=inverse_PSNR)
    history = model.fit(train, epochs=epochs, validation_data=val, verbose=2)
